In [ ]:
"""
__SPICE OBJECT NAIF-ID CODES__
-3 : MOM, MARS ORBITER MISSION
-5 : AKATSUKI,PLC,VCO,
0  : SSB, SOLAR SYSTEM BARYCENTER
399004 : USUDA
399932 : IDSN32
"""

In [ ]:
# GEOMETRIES DONE

import numpy as np 
import matplotlib.pyplot as plt
import spiceypy as spice
import pandas as pd
import warnings
import itertools
warnings.filterwarnings("ignore")

spice_env = ('./kernels/requirements.txt')
spice.furnsh(spice_env) 
spice.ktotal('ALL')
#-----------------------------------------------------------------------------------------------------------------------------
GM_Venus    = 3.2468e14
GM_Earth    = 3.986e14
GM_Sun      = 1.327e20
GM_Mars     = 4.2828e13
f_sc        = 2296368750.0 
c           = 299792458.0
delta_T     = 1.0
AU          = 1.496e+11
solar_radii = 6.95700e+8
#-----------------------------------------------------------------------------------------------------------------------------
def orthoprojection(ax,ay,az,bx,by,bz,cx,cy,cz):
    abx = bx-ax
    aby = by-ay
    abz = bz-az
    acx = cx-ax
    acy = cy-ay
    acz = cz-az
    t = (abx*acx + aby*acy + abz*acz)/(abx*abx + aby*aby + abz*abz)
    px = ax + t*abx
    py = ay + t*aby
    pz = az + t*abz
    return px,py,pz
#-----------------------------------------------------------------------------------------------------------------------------
step = 1
utc = ['Oct 14, 2021']
et1 = spice.str2et(utc[0])

# T2 = et1+delta_T
T2  = [et1+ x for x in range(0,step)]
_, lightime = spice.spkezr('IDSN32',T2,'J2000','NONE','MOM')
T1  = [m-n  for m,n in zip(T2,lightime)]

print(np.shape(T1))
print("T1 : ",T1)
print("\n")

print(np.shape(T1))
print("T2 : ",T2)
print("\n")

for (t1,t2) in zip(T1,T2):
    print("#--------------------------------------------------------------------------------------------------------------------")
    print("t1: ",t1,"t2:",t2)
    print("#--------------------------------------------------------------------------------------------------------------------")

    MOM_state_T1_S , _= spice.spkezr('MOM',t1,'J2000', 'NONE', 'SUN')
    print("Shapes for state for MOM : SME",np.shape(MOM_state_T1_S))
    MOM_state_T1_S    = np.array(MOM_state_T1_S)
    print("#--------------------------------------------------------------------------------------------------------------------")

    IDSN_state_T2_S, _ = spice.spkezr('IDSN32',t2,'J2000', 'NONE', 'SUN')
    print("Shapes for state for IDSN : SME",np.shape(IDSN_state_T2_S))
    IDSN_state_T2_S    = np.array(IDSN_state_T2_S)
    print("#--------------------------------------------------------------------------------------------------------------------")
    v_IDSN = IDSN_state_T2_S[3:6]*1000
    v_MOM  = MOM_state_T1_S[3:6]*1000
    print("Shapes for velocities : ", np.shape(v_IDSN), np.shape(v_MOM))
    # print("#--------------------------------------------------------------------------------------------------------------------")
    # print("v_MOM : ",v_MOM)
    # print("#----------------------------------------------------------")
    # print("v_IDSN :",v_IDSN)
    print("#--------------------------------------------------------------------------------------------------------------------")
    r_MOM_S   = (MOM_state_T1_S[0:3]*1000)
    print("Shapes for Position for MOM : ", np.shape(r_MOM_S))
    print("#--------------------------------------------------------------------------------------------------------------------")

    r_IDSN_S   = (IDSN_state_T2_S[0:3]*1000)
    print("Shapes for Position for IDSN : ", np.shape(r_IDSN_S))
    print("#--------------------------------------------------------------------------------------------------------------------")

    offset_point = orthoprojection((MOM_state_T1_S[0]*1000),(MOM_state_T1_S[1]*1000),(MOM_state_T1_S[2]*1000),(IDSN_state_T2_S[0]*1000),(IDSN_state_T2_S[1]*1000),(IDSN_state_T2_S[2]*1000),0,0,0)
    print("Shapes for Offset point : ",np.shape(offset_point))
    print("Coordinates for offset point : ",offset_point)

    x = offset_point[0]
    y=offset_point[1]
    z = offset_point[2]
    coords = [x,y,z]
    RRsun = np.sqrt((x**2)+(y**2)+(z**2))
    offset_R = RRsun/solar_radii
    offset_rrsun = offset_R
    print("Solar offset in Solar radii : ",offset_rrsun)
    print("#--------------------------------------------------------------------------------------------------------------------")

    r_probe_solar_offset_abs =np.sqrt((r_MOM_S[0]-offset_point[0])**2 + (r_MOM_S[1]-offset_point[1])**2 + (r_MOM_S[2]-offset_point[2])**2)
    print("Distance between the probe and the point of closest approach to the Sun in AU : ",r_probe_solar_offset_abs/AU) 

    r_earth_solar_offset_abs =np.sqrt((r_IDSN_S[0]-offset_point[0])**2 + (r_IDSN_S[1]-offset_point[1])**2 + (r_IDSN_S[2]-offset_point[2])**2)
    print("Distance between the Earth and the point of closest approach to the Sun in AU : ",r_earth_solar_offset_abs/AU)
    print("#--------------------------------------------------------------------------------------------------------------------")

    r_MOM_S_N = np.linalg.norm(r_MOM_S) #[np.linalg.norm(rS)for rS in r_MOM_S]
    r_MOM_S_abs   = np.array(r_MOM_S_N)
    print("Shapes for Position for MOM after normalizing : ", np.shape(r_MOM_S_abs))
    print("#--------------------------------------------------------------------------------------------------------------------")
    print("r_MOM_S : ",r_MOM_S_abs/AU)
    print("#--------------------------------------------------------------------------------------------------------------------")

    r_IDSN_S_N = np.linalg.norm(r_IDSN_S)# [np.linalg.norm(rIS) for rIS in r_IDSN_S]
    r_IDSN_S_abs   = np.array(r_IDSN_S_N)
    print("Shapes for Position for IDSN after normalizing : ", np.shape(r_IDSN_S_abs))
    print("#--------------------------------------------------------------------------------------------------------------------")
    print("r_IDSN_S : ",r_IDSN_S_abs/AU)
    print("#--------------------------------------------------------------------------------------------------------------------")

    rME_abs =np.sqrt((r_MOM_S[0]-r_IDSN_S[0])**2 + (r_MOM_S[1]-r_IDSN_S[1])**2 + (r_MOM_S[2]-r_IDSN_S[2])**2)
    print("Earth Probe Distance in m : ",rME_abs)
    print("Earth Probe Distance in AU : ",rME_abs/AU)
    print("#--------------------------------------------------------------------------------------------------------------------")

    Card = np.arccos(((r_MOM_S_abs/AU)**2 + (r_IDSN_S_abs/AU)**2 - (rME_abs/AU)**2)/(2*(r_MOM_S_abs/AU)*(r_IDSN_S_abs/AU)))
    Card = np.nan_to_num(Card)
    print("ESP Angle in Radians : ",Card)
    print("#--------------------------------------------------------------------------------------------------------------------")

    Boat = np.arccos(((r_IDSN_S_abs/AU)**2 - (r_MOM_S_abs/AU)**2 + (rME_abs/AU)**2)/(2*(rME_abs/AU)*(r_IDSN_S_abs/AU)))
    Boat = np.nan_to_num(Boat)
    print("SEP Angle in Radians : ",Boat)
    print("#--------------------------------------------------------------------------------------------------------------------")
    print("GEOMETRIES DONE")
    print("#--------------------------------------------------------------------------------------------------------------------")

geometries = {'T1_MOM':T1,
                  'T2_IDSN':T2,
                  'r_IDSN_S_AU':r_IDSN_S_abs/AU,
                  'r_MOM_S_AU':r_MOM_S_abs/AU,
                  'offset_rsun':offset_rrsun,
                  'probe_to_pt_of_closest_approach_dist_AU':r_probe_solar_offset_abs/AU,
                  'earth_to_pt_of_closest_approach_dist_AU':r_earth_solar_offset_abs/AU,
                  'Earth_probe_dist_AU':rME_abs/AU,
                  'Earth_Sun_Probe_angle':Card,
                  'Sun_Earth_Probe_angle':Boat,
                  }

df = pd.DataFrame(geometries)
df.to_csv('geometries/geometries_test.txt',sep='\t',index=False)
print("Done.")